In [17]:
import numpy as np
import pandas as pd

#LOADING DATA FROM TRAIN AND TEST DATASET

df1=pd.read_csv('/content/drive/MyDrive/SML_Project/train.csv')

df2=pd.read_csv('/content/drive/MyDrive/SML_Project/test.csv')

df1

,ID,n0,n1,n2,n3,n4,n5,n6,n7,n8,...,n4087,n4088,n4089,n4090,n4091,n4092,n4093,n4094,n4095,category
0,0,0.000000,0.000000,1.272801,0.290501,0.581446,0.000000,0.000000,0,0.000000,...,1.645888,0.869640,0.302432,0.953719,0.022545,0.498048,0.000000,0.034988,0.692382,Orange_Ripe
1,1,0.000000,0.000000,1.542096,0.000000,0.896557,0.049978,0.000000,0,0.117847,...,1.504220,0.622686,0.588427,0.524415,0.305426,0.386204,0.000000,0.000000,0.668196,Banana_Ripe
2,2,0.000000,0.000000,1.098595,0.571866,0.500355,0.000000,0.000000,0,0.493137,...,1.169341,0.913239,0.064404,0.531270,0.000000,0.471604,0.000000,0.000000,0.658250,Mango_Raw
3,3,0.000000,0.101666,1.159194,0.599216,0.893206,0.000000,0.200139,0,0.645675,...,0.560686,1.243676,0.432523,0.701881,0.000000,0.589985,0.000000,0.000000,0.591165,Leeche_Raw
4,4,0.000000,0.000000,1.178603,0.362568,0.577602,0.000000,0.000000,0,0.079862,...,1.206032,0.736831,0.345906,0.878515,0.119000,0.261441,0.000000,0.000000,0.458905,Mango_Ripe
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1211,1211,0.000000,0.364963,0.770978,0.570945,0.996824,0.066661,0.000000,0,0.727390,...,0.317000,0.844568,0.084291,0.399840,0.225693,0.214918,0.000000,0.000000,0.356400,Strawberry_Ripe
1212,1212,0.083656,0.111407,1.753287,0.000000,1.197256,0.117919,0.000000,0,0.000000,...,1.114398,0.659943,0.000000,0.457870,0.405808,0.000000,0.342428,0.000000,0.019390,Banana_Ripe
1213,1213,0.000000,0.000000,1.545725,0.000000,0.842485,0.000000,0.000000,0,0.087607,...,1.490054,0.883849,0.589065,0.644103,0.194427,0.426330,0.000000,0.000000,0.764020,Banana_Ripe
1214,1214,0.000000,0.000000,1.194037,0.969926,0.499340,0.055789,0.000000,0,0.221447,...,0.987144,0.645109,0.113367,0.877268,0.000000,0.175457,0.000000,0.000000,0.367878,Guava_Ripe


In [18]:
# y_train will consist of the category only(in the X_train)

y_train = df1.iloc[:, -1]

#X_train will consist of all the features(remove the ID and category from the dataset)
df1=df1.drop(['ID', 'category'], axis=1)

X_train= df1.to_numpy()

#Remove ID from the test_data as well(only category should remain)
print(df2.shape)
test_data = df2.drop('ID',axis=1)
y_train


(415, 4097)


0           Orange_Ripe
1           Banana_Ripe
2             Mango_Raw
3            Leeche_Raw
4            Mango_Ripe
             ...       
1211    Strawberry_Ripe
1212        Banana_Ripe
1213        Banana_Ripe
1214         Guava_Ripe
1215         Guava_Ripe
Name: category, Length: 1216, dtype: object

In [19]:
#Remove the features which are acting as noise(Dimensionality Reduction)--->Feature Selection(Variance thresholding)

from sklearn.feature_selection import VarianceThreshold

# Threshold value
threshold = 0.01

vt = VarianceThreshold(threshold=threshold)

X_train = vt.fit_transform(X_train)

# Remove corresponding features from the test data
test_data = vt.transform(test_data)


/usr/local/lib/python3.9/dist-packages/sklearn/base.py:432: UserWarning: X has feature names, but VarianceThreshold was fitted without feature names
  warnings.warn(


In [20]:
# Used clustering algorithm--> KMeans to have cluster labels as additional features

from sklearn.cluster import KMeans

kmeans = KMeans(n_clusters=25, random_state=42)
train_clusters = kmeans.fit_predict(X_train)

# Adding cluster label to the training data as well as the test data

X_train = np.column_stack((X_train,train_clusters))

test_data = np.column_stack((test_data,kmeans.fit_predict(test_data)))

/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/sklearn/cluster/_kmeans.py:870: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  warnings.warn(


In [21]:
from sklearn.decomposition import PCA

pca = PCA(n_components=400)
X_train = pca.fit_transform(X_train)

X_train=np.array(X_train)

test_data=pca.fit_transform(test_data)
test_data=np.array(test_data)


In [22]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train= scaler.fit_transform(X_train)
X_train.shape

test_data= scaler.fit_transform(test_data)
test_data.shape




(415, 400)

In [23]:
# Outlier detection algorithm used-->Isolation Forest to remove outliers. 

from sklearn.ensemble import IsolationForest

iso_forest = IsolationForest(random_state=42,contamination=0.1)

iso_forest.fit(X_train)
outliers = iso_forest.predict(X_train)

#Removing those samples from the dataset which are outliers
X_train = X_train[outliers == 1]

y_train = y_train[outliers == 1]

#To know the final number of samples in X_train(after some outliers are removed)
X_train.shape

(1094, 400)

In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier

#Classification using logistic regression and then ensemble method(Bagging classifier)
lr = LogisticRegression(random_state=42)

bagging_lr = BaggingClassifier(estimator=lr)

bagging_lr.fit(X_train, y_train)

y_test_pred = bagging_lr.predict(test_data)

#Cross validation is used(5 fold)
scores = cross_val_score(bagging_lr, X_train, y_train, cv=5, scoring='accuracy')

print(f'Cross-validation scores: {scores}')
print(f'Mean score: {np.mean(scores)}')

submission = pd.DataFrame({'ID': df2['ID'], 'category': y_test_pred})
submission.to_csv('submission.csv', index=False)


Cross-validation scores: [0.74885845 0.79452055 0.7260274  0.73972603 0.69724771]
Mean score: 0.7412760253026685
